In [ ]:
!pip install --upgrade timm
!pip install --upgrade efficientnet-pytorch

In [ ]:
%cd "/content/drive/My Drive/Colab Notebooks/nishika/Japanese Painting Classification"

In [ ]:
# ライブラリ読み込み
import os
import glob
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, f1_score
from PIL import Image
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
get_ipython().run_line_magic('matplotlib', 'inline')

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torchvision import transforms
from efficientnet_pytorch import EfficientNet
import timm

In [ ]:
SEED = 42
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed_everything(SEED)
current_device = torch.cuda.current_device()
print("Device:", torch.cuda.get_device_name(current_device))

In [7]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
TRAIN_DIR = "./data/train/"
TEST_DIR = "./data/test/"
USE_PRETRAINED = True
EPOCHS = 30
N_SPLITS = 5

In [8]:
# データのオーギュメンテーションと前処理
class ImageTransform():
    def __init__(self):
        self.data_transform = {
            "train": transforms.Compose([
                transforms.Resize(256),  # リサイズ
                transforms.CenterCrop(256),  # 画像中央を切り取り
                transforms.RandomHorizontalFlip(),  # 半分の確率で画像を左右反転させる
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # 標準化
            ]),
            "test": transforms.Compose([
                transforms.Resize(256),  # リサイズ
                transforms.CenterCrop(256),  # 画像中央を切り取り
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # 標準化
            ]),
            "test_tta": transforms.Compose([
                transforms.Resize(256),  # リサイズ
                transforms.CenterCrop(256),  # 画像中央を切り取り
                transforms.RandomHorizontalFlip(p=1.0),
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # 標準化
            ])
        }

    def __call__(self, image, phase="trian"):
        return self.data_transform[phase](image)

In [9]:
# Dataset作成
class KaokoreDataset(data.Dataset):
    def __init__(self, df_data, root_dir, transform=None, phase='train'):
        self.root_dir = root_dir
        self.image_list = df_data["image"].tolist()
        self.label_list = df_data["gender_status"].tolist()
        self.transform = transform
        self.phase = phase
        
    def __len__(self):
        return len(self.image_list)
    
    def __getitem__(self, index):
        image_path = self.image_list[index]
        image = Image.open(self.root_dir + image_path)
        image = self.transform(image, self.phase)

        label = self.label_list[index]
        return image, label

In [10]:
class ClassifierB7ep(nn.Module):
    def __init__(self, num_classes=8):
        super().__init__()
        self.model = EfficientNet.from_pretrained("efficientnet-b7", num_classes=num_classes)
        self.model = self.model.to(DEVICE)

    def forward(self, input):
        output = self.model(input)
        return output

class ClassifierB7timm(nn.Module):
    def __init__(self, num_classes=8):
        super().__init__()
        self.model = timm.create_model("tf_efficientnet_b7_ns", num_classes=num_classes)
        self.model = self.model.to(DEVICE)

    def forward(self, input):
        output = self.model(input)
        return output

class ClassifierB8timm(nn.Module):
    def __init__(self, num_classes=8):
        super().__init__()
        self.model = timm.create_model("tf_efficientnet_b8", num_classes=num_classes)
        self.model = self.model.to(DEVICE)

    def forward(self, input):
        output = self.model(input)
        return output

In [ ]:
DIR_B7_EP = "./models/model_1_seed42_efficientnet_pytorch_efficientnet_b7/"
DIR_B7_TIMM = "./models/model_2_seed43_timm_tf_efficientnet_b7_ns/"
DIR_B8_TIMM = "./models/model_3_seed44_timm_tf_efficientnet_b8/"

models = []
for fold in range(N_SPLITS):
    model = ClassifierB7ep()
    model.model.load_state_dict(torch.load(DIR_B7_EP + f"best_model_{fold}.pt"))
    model.eval()
    models.append(model)

for fold in range(N_SPLITS):
    model = ClassifierB7timm()
    model.model.load_state_dict(torch.load(DIR_B7_TIMM + f"best_model_{fold}.pt"))
    model.eval()
    models.append(model)
    print("Loaded pretrained weights")

for fold in range(N_SPLITS):
    model = ClassifierB8timm()
    model.model.load_state_dict(torch.load(DIR_B8_TIMM + f"best_model_{fold}.pt"))
    model.eval()
    models.append(model)
    print("Loaded pretrained weights")

In [12]:
df_test = pd.read_csv("./data/sample_submission.csv")
test_dataset = KaokoreDataset(df_test, TEST_DIR, ImageTransform(), phase="test")
test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size = 64, shuffle=False
)

test_dataset_tta = KaokoreDataset(df_test, TEST_DIR, ImageTransform(), phase="test_tta")
test_dataloader_tta = torch.utils.data.DataLoader(
    test_dataset, batch_size = 64, shuffle=False
)

In [ ]:
with torch.no_grad():
    progress = tqdm(zip(test_dataloader, test_dataloader_tta), total=len(test_dataloader))
    final_output = []

    for (input, _), (input_tta, _) in progress:
        input, input_tta = input.to(DEVICE), input_tta.to(DEVICE)
        outputs = []
        for model in models:
            model.eval()
            output = model(input)
            outputs.append(output)
        for model in models:
            model.eval()
            output = model(input_tta)
            outputs.append(output)

        outputs = sum(outputs) / len(outputs)
        outputs = torch.softmax(outputs, dim=1).cpu().detach().tolist() 
        outputs = np.argmax(outputs, axis=1)

        final_output.extend(outputs)

In [ ]:
sample = pd.read_csv("./data/sample_submission.csv")
sample.loc[:, "gender_status"] = final_output
sample.to_csv("./outputs/submission_emsemble_tta.csv", index=False)

In [ ]:
sample.head()